In [1]:
import polars as pl 
import numpy as np
cov = 0.5001


elongates = pl.read_csv(f"output/{cov}/{cov}_elongates.csv", infer_schema_length =10000)

def gini_coefficient(x):
    
    """
    Compute Gini coefficient of array of values
    Source : https://stackoverflow.com/questions/39512260/calculating-gini-coefficient-in-python-numpy by Ulf Aslak
    """

    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        
        print(i,xi)
        diffsum += np.sum(np.abs(xi - x[i:]))
        print(diffsum)
    return diffsum / (len(x)**2 * np.mean(x))

In [161]:
import polars as pl
seuil = 4

elongates = pl.read_csv(f"output/{cov}/{cov}_elongates.csv", infer_schema_length =10000)

########
# Get the clusters that are elongated in Scer but not in Sbay
# Knowing well that such clusters include clusters without Sbay, or only with Scer or any other scenario 
########

Nter_scer_conditions = ((pl.col("species") == "Scer_NCBI") & (abs(pl.col("max_Nter") - pl.col("Nter_nb_aa")) < seuil) & (pl.col("max_Nter") >= 10))
Nter_sbay_conditions = ((pl.col("species") == "Sbay") & (pl.col("Nter_nb_aa") > seuil))

Nter_scer_clusters = set(elongates.filter(Nter_scer_conditions)["cluster_name"].to_list())
Nter_sbay_clusters = set(elongates.filter(Nter_sbay_conditions)["cluster_name"].to_list())

Nter_clusters = elongates.filter(pl.col("cluster_name").is_in(Nter_scer_clusters- Nter_sbay_clusters))



Cter_scer_conditions = ((pl.col("species") == "Scer_NCBI") & (abs(pl.col("max_Cter") - pl.col("Cter_nb_aa")) < seuil) & (pl.col("max_Cter") >= 10))
Cter_sbay_conditions = ((pl.col("species") == "Sbay") & (pl.col("Cter_nb_aa") > seuil))

Cter_scer_clusters = set(elongates.filter(Cter_scer_conditions)["cluster_name"].to_list())
Cter_sbay_clusters = set(elongates.filter(Cter_sbay_conditions)["cluster_name"].to_list())

Cter_clusters = elongates.filter(pl.col("cluster_name").is_in(Cter_scer_clusters- Cter_sbay_clusters))


########
# From those clusters, we want to keep only the ones that have Scer AND Sbay
########


# Nter
scer_df_nter = set(Nter_clusters.filter(pl.col("species") == "Scer_NCBI")["cluster_name"].to_list())
sbay_df_nter = set(Nter_clusters.filter(pl.col("species") == "Sbay")["cluster_name"].to_list())
common = scer_df_nter.intersection(sbay_df_nter)

full_filtered_Nter = Nter_clusters.filter(pl.col("cluster_name").is_in(common)).sort("cluster_name")

# Cter
scer_df_cter = set(Cter_clusters.filter(pl.col("species") == "Scer_NCBI")["cluster_name"].to_list())
sbay_df_cter = set(Cter_clusters.filter(pl.col("species") == "Sbay")["cluster_name"].to_list())
common = scer_df_cter.intersection(sbay_df_cter)

full_filtered_Cter = Cter_clusters.filter(pl.col("cluster_name").is_in(common)).sort("cluster_name")


# Question : are there some clusters that correspond to both Nter and Cter elongation criteria ?

# Answer : 

# full_filtered_Nter.filter(pl.col("cluster_name").is_in(full_filtered_Cter["cluster_name"].unique().to_list()))

cluster_size,cluster_name,seq_id,species,sequence,sequence_length,Nter_dashes,Cter_dashes,max_Nter,max_Cter,Nter_elongate,Nter_gaps,Nter_gap_openings,Nter_nb_aa,Nter_elongate_length,Cter_elongate,Cter_gaps,Cter_gap_openings,Cter_nb_aa,Cter_elongate_length,Nter_ratio,Cter_ratio,is_max_Nter,is_max_Cter,is_min_Nter,is_min_Cter,Meth_after_Nter,Nter_event_ID,Cter_event_ID,Nter_events,Cter_events
i64,str,str,str,str,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,str,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,str,i64,i64,i64,i64
5,"""cluster_n2372""","""Sbay_8.46""","""Sbay""","""--------------…",636,59,10,59,10,null,0,0,0,0,null,0,0,0,0,0.0,0.0,0,0,1,1,"""1""",0,0,1,1
5,"""cluster_n2372""","""Sarb_08G00400.…","""Sarb""","""MVDDSSYLTPHETA…",636,0,0,59,10,"""MVDDSSYLTPHETA…",0,0,59,59,"""PHQCHTTSQL""",0,0,10,10,0.092767,0.015723,1,1,0,0,"""0""",1,1,1,1
5,"""cluster_n2372""","""Skud_8.38""","""Skud""","""MVDDSSYLTPHETA…",636,0,0,59,10,"""MVDDSSYLTPHETA…",0,0,59,59,"""SHNRHTTSEL""",0,0,10,10,0.092767,0.015723,1,1,0,0,"""1""",1,1,1,1
5,"""cluster_n2372""","""rna-NM_0011790…","""Scer_NCBI""","""MVDDSNYLTPHETA…",636,0,0,59,10,"""MVDDSNYLTPHETA…",0,0,59,59,"""FYNRHTSPQL""",0,0,10,10,0.092767,0.015723,1,1,0,0,"""0""",1,1,1,1
5,"""cluster_n2372""","""rna-XM_0339107…","""Spar_NCBI""","""MVDDSNYLTPHETA…",636,0,0,59,10,"""MVDDSNYLTPHETA…",0,0,59,59,"""SYNRHTTPQL""",0,0,10,10,0.092767,0.015723,1,1,0,0,"""1""",1,1,1,1


In [ ]:
from utils.handle_UTRs import get_extended_UTRs
import os

cov = 0.5
current_path = f"/home/simon.herman/Bureau/Gits/Elongates/work/{cov}"

os.mkdir(f"{current_path}/local_align_files")


dataframes = {
    "Nter": full_filtered_Nter,
    "Cter": full_filtered_Cter,
}

for side in ["Nter", "Cter"]:

    df = dataframes[side]

    os.mkdir(f"{current_path}/local_align_files/{side}")

    for cluster, sequences in df.groupby("cluster_name"):

        os.mkdir(f"{current_path}/local_align_files/{side}/{cluster}")

        scer_length = sequences.filter(pl.col("species") == "Scer_NCBI")[f"{side}_nb_aa"].max() # Maybe several Scer sequences in the cluster, we take the longest elongate

        input_dict = dict()

        for sequence in sequences.iter_rows(named = True):

            if scer_length - sequence[f"{side}_nb_aa"] >= 10:

                input_dict[sequence["seq_id"]] = sequence["Nter_nb_aa"]
                os.mkdir(f"{current_path}/local_align_files/{side}/{cluster}/{sequence['seq_id']}")
                os.mkdir(f"{current_path}/local_align_files/{side}/{cluster}/{sequence['seq_id']}/nucleotide")
                os.mkdir(f"{current_path}/local_align_files/{side}/{cluster}/{sequence['seq_id']}/protein")

                dict_ = custom_target_elongate()

                

                print(sequence["seq_id"], sequence[f"{side}_nb_aa"])
            

            






In [187]:
def modular_UTRs(input_dict): 

    

['cluster_n5063',
 'cluster_n4982',
 'cluster_n1640',
 'cluster_n2529',
 'cluster_n6341',
 'cluster_n2063',
 'cluster_n3835',
 'cluster_n1590',
 'cluster_n1011',
 'cluster_n428',
 'cluster_n536',
 'cluster_n3239',
 'cluster_n5430',
 'cluster_n4167',
 'cluster_n2058',
 'cluster_n1854',
 'cluster_n334',
 'cluster_n6451',
 'cluster_n4767',
 'cluster_n1374',
 'cluster_n6345',
 'cluster_n6586',
 'cluster_n6633',
 'cluster_n4133',
 'cluster_n4608',
 'cluster_n1857',
 'cluster_n4128',
 'cluster_n6531',
 'cluster_n1164',
 'cluster_n3236',
 'cluster_n4680',
 'cluster_n5062',
 'cluster_n1179',
 'cluster_n5197',
 'cluster_n6457',
 'cluster_n3110',
 'cluster_n6026',
 'cluster_n5005',
 'cluster_n6209',
 'cluster_n4565',
 'cluster_n5085',
 'cluster_n896',
 'cluster_n4416',
 'cluster_n3375',
 'cluster_n5321',
 'cluster_n922',
 'cluster_n3036',
 'cluster_n5751',
 'cluster_n4470',
 'cluster_n6388',
 'cluster_n2232',
 'cluster_n5633',
 'cluster_n2372',
 'cluster_n3883',
 'cluster_n3715',
 'cluster_n4262